In [1]:
from openai import OpenAI
import os
import configparser
import json
import tqdm
import time
import random

api_key = ''
with open('../openai.txt', 'r') as file:
    api_key = file.read().replace('\n', '')

client = OpenAI(api_key=api_key)

In [2]:
def list_models():
    """
    List all available models in the OpenAI API.
    """
    models = client.models.list()
    print("Available models:")

    model_ids = []
    for model in models.data:
        model_ids.append(model.id)

    model_ids.sort()
    for model_id in model_ids:
        print(model_id)

# list_models(client)

In [3]:
def costs(prompt_tokens, completion_tokens):
    """
    Calculate the costs based on the number of tokens used.
    Args:
        prompt_tokens (int): The number of prompt tokens used.
        completion_tokens (int): The number of completion tokens used.
    Returns:
        float: The total cost in dollars.
    """
    if model_used == 'gpt-4o-mini':
        price_per_1m_input_tokens = 0.15
        price_per_1m_output_tokens = 0.6
    elif model_used == 'gpt-4o':
        price_per_1m_input_tokens = 2.5
        price_per_1m_output_tokens = 10
    else:
        raise ValueError(
            "Model not set or not supported for cost calculation.")
    total_cost = ((prompt_tokens / 1000000) * price_per_1m_input_tokens) + \
        ((completion_tokens / 1000000) * price_per_1m_output_tokens)
    return total_cost

In [ ]:
def file_upload(location):
    """
    Upload a file to the OpenAI API.
    Args:
        location (str): The file path to upload.
    Returns:
        file: The uploaded file object.
    """
    with open(location, "rb") as file:
        file = client.files.create(
            file=file,
            purpose="user_data"
        )

    print("File uploaded successfully.")
    return file


def file_delete(file_id):
    """
    Delete a file from the OpenAI API.
    Args:
        file_id (str): The ID of the file to delete.
    Returns:
        bool: True if the file was deleted successfully, False otherwise.
    """
    try:
        client.files.delete(file_id)
        print("File deleted successfully.")
        return True
    except Exception as e:
        print(f"Error deleting file: {e}")
        return False


def list_files():
    """
    List all files uploaded to the OpenAI API.
    """
    files = client.files.list()
    print("Available files:")
    for file in files.data:
        print(file.id, file.filename, file.status)

    return

In [ ]:
def load_prompts(file_path):
    config = configparser.ConfigParser(allow_no_value=True, delimiters=("="))
    config.optionxform = str
    config.read(file_path)

    prompts = {}
    for section in config.sections():
        prompts[section] = "\n".join(
            [f"{key}{value}" for key, value in config.items(section)])
    return prompts


def replace_variables(string, **kwargs):
    return string.format(**kwargs)


def read_files(file_paths):
    json_file = None
    txt_file = None
    for filename in os.listdir(file_paths):
        if filename.endswith('.json'):
            json_file = filename
        elif filename.endswith('.txt'):
            txt_file = filename

    if json_file is None or txt_file is None:
        raise ValueError("Required files not found in the directory.")

    settings = {}
    prompts = {}

    with open(os.path.join(file_paths, json_file), 'r') as f:
        settings = json.load(f)

    prompts = load_prompts(os.path.join(file_paths, txt_file))

    return settings, prompts

In [ ]:
def llm_call(assistant_history, user_history):
    """
    Call the OpenAI API to get a response based on the provided system and user prompts.
    Args:
        assistant_history (str): The assistant's previous responses.
        user_history (str): The user's previous responses.
    Returns:
        assistant_history (str): New assistant's history.
        user_history (str): New user's history.
        prompt_tokens (int): The number of prompt tokens used.
        completion_tokens (int): The number of completion tokens used.
    """
    retries = 0
    while True:
        try:
            response = client.chat.completions.create(
                model=model_used,
                messages=assistant_history,
                max_completion_tokens=500
            )
            break
        except Exception as e:
            print(f"Error: {e}")
            retries += 1
            backoff_time = (2 ** retries) + random.random()
            print(f"Retrying in {backoff_time:.2f} seconds...")
            time.sleep(backoff_time)

    prompt_tokens = response.usage.prompt_tokens
    completion_tokens = response.usage.completion_tokens

    global global_prompt_tokens, global_completion_tokens
    global_prompt_tokens += response.usage.prompt_tokens
    global_completion_tokens += response.usage.completion_tokens

    assistant_history.append(
        {"role": "assistant", "content": response.choices[0].message.content})
    user_history.append(
        {"role": "user", "content": response.choices[0].message.content})

    return assistant_history, user_history, prompt_tokens, completion_tokens

In [7]:
def store_history(student_history, teacher_history, metadata):
    """
    Store the history in a JSON file.
    Args:
        student_history (list): The student-teacher's history.
        teacher_history (list): The teacher-educator's history.
        metadata (dict): Additional metadata to store.
    """
    all_dialogues = []

    if os.path.exists(results):
        with open(results, "r") as file:
            try:
                all_dialogues = json.load(file)
                if not isinstance(all_dialogues, list):
                    all_dialogues = [all_dialogues]
            except json.JSONDecodeError:
                # If the file is empty or invalid, start with an empty list
                all_dialogues = []

    # Append to the results file
    dialogue_results = {
        "student": student_history,
        "teacher": teacher_history,
        "metadata": metadata
    }

    all_dialogues.append(dialogue_results)

    with open(results, "w") as file:
        json.dump(all_dialogues, file, indent=4)

In [8]:
def add_tokens(tpt, tct, ttt, prompt_tokens, completion_tokens):
    tpt += prompt_tokens
    tct += completion_tokens
    ttt += prompt_tokens + completion_tokens
    return tpt, tct, ttt

In [ ]:
def socratic_dialogue(settings, prompts, sources, attempt, max_iterations):
    """
    Conduct a Socratic dialogue between a Student-Teacher and a Teacher-Educator.
    Args:
        settings (dict): The settings for the dialogue.
        prompts (dict): The prompts for the dialogue.
        sources (list): Additional sources to use in the dialogue.
        attempt (int): The attempt number.
        max_iterations (int): The maximum number of iterations for the dialogue.
    """
    # LLM token count
    tpt = 0
    tct = 0
    ttt = 0

    iteration_count = 0

    # Add the system prompts
    st_history = [
        {
            "role": "system",
            "content": prompts["student_teacher_start"]
        }
    ]
    te_history = [
        {
            "role": "system",
            "content": prompts['teacher_educator']
        }
    ]

    # If materials are set, use them in the dialogue
    if settings['materials']:
        materials = []
        for source in sources:
            materials.append({"type": "file", "file": {"file_id": source}})

        st_history.append({
            "role": "user",
            "content": materials
        })
        te_history.append({
            "role": "user",
            "content": materials
        })

    for i in range(max_iterations):
        iteration_count += 1
        if i == 0:
            st_history, te_history, pt, ct = llm_call(
                st_history, te_history)

            # Replace the starting system prompt of the Student Teacher
            st_history[0] = {"role": "system",
                             "content": prompts['student_teacher']}
        else:
            st_history, te_history, pt, ct = llm_call(
                st_history, te_history)

        tpt, tct, ttt = add_tokens(tpt, tct, ttt, pt, ct)

        te_history, st_history, pt, ct = llm_call(
            te_history, st_history)
        tpt, tct, ttt = add_tokens(tpt, tct, ttt, pt, ct)

        # Check if the teacher thinks the question is sufficient
        if "Great question!" in te_history[-1]["content"]:
            break
    else:
        # If the loop completes without breaking, it means the iteration limit was reached
        st_history, te_history, pt, ct = llm_call(
            st_history, te_history)
        tpt, tct, ttt = add_tokens(tpt, tct, ttt, pt, ct)

    metadata = {
        "iterations": iteration_count,
        "prompt_tokens": tpt,
        "completion_tokens": tct,
        "total_tokens": ttt,
        "attempt": attempt,
        "model_used": model_used,
        "student_level": settings['student_level'],
        "materials": settings['materials']
    }

    store_history(st_history, te_history, metadata)

In [10]:
def experiment(directory, attempts, max_iterations, topic, concepts, student_level, sources):
    if not topic or not concepts or not student_level or not sources or not directory or not attempts:
        raise ValueError("All parameters must be provided.")

    if os.path.exists(results):
        os.remove(results)
        with open(results, 'w') as f:
            json.dump([], f)

    print("Starting the generation...")

    number = 0
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)
        if not os.path.isdir(subdir_path):
            continue

        settings, prompts = read_files(subdir_path)

        # Perform the replacements
        prompts = {key: replace_variables(value, topic=topic, concepts=concepts,
                                          student_level=student_level) for key, value in prompts.items()}

        # Do x attempts
        for i in tqdm.tqdm(range(attempts), desc=f"Processing {subdir}"):
            socratic_dialogue(settings, prompts, sources,
                              number, max_iterations)
            number += 1

    print("All combinations have been processed.")

# Materials
```python
topic = "Basics of how the internet works"
concepts = """
 - Decentralization of the internet
 - Servers, datacenters and routers
 - Server vs client
 - Data packets
 - IP addresses
"""
student_level = "8th or 9th grade"
sources = ["file-R2qW94jP5GW8PE3JMzVf2f", "file-2tJuwTnPH9Pf96vDteRQ5C"]
```

In [ ]:
model_used = "gpt-4o-mini"

global_prompt_tokens = 0
global_completion_tokens = 0

# Materials
topic = "Basics of how the internet works"
concepts = """
 - Decentralization of the internet
 - Servers, datacenters and routers
 - Server vs client
 - Data packets
 - IP addresses
"""
student_level = "8th or 9th grade"
sources = ["file-R2qW94jP5GW8PE3JMzVf2f", "file-2tJuwTnPH9Pf96vDteRQ5C"]

# Experiment settings
directory = 'Fixed10iter/Prompts'  # Directory containing directories with prompts
results = "Fixed10iter/fixed10iter.json"  # Name of the results file
attempts = 10  # Attempts per each combination
max_iterations = 10  # Maximum iterations for Socratic dialogue

experiment(directory, attempts, max_iterations,
           topic, concepts, student_level, sources)

Starting the generation...


Processing dww: 100%|██████████| 10/10 [07:48<00:00, 46.87s/it]

All combinations have been processed.


Experiments:

Prompt combinations:
- dialogue + student level + materials  - DLM
- dialogue + student level + without    - DLW
- dialogue + without + materials        - DWM
- dialogue + without + without          - DWW

Run only for the 10 iterations, then no dialogue is only the first answer, and 5 iterations can be taken from this as well for comparison.

In [12]:
print("Total prompt tokens:", global_prompt_tokens)
print("Total completion tokens:", global_completion_tokens)
print(costs(global_prompt_tokens, global_completion_tokens))

Total prompt tokens: 13032491
Total completion tokens: 92622
2.01044685


In [ ]:
with open(results, 'r') as f:
    data = json.load(f)

# print the number of how many times each iteration was used
iterations_count = {}
for result in data:
    iterations = result['metadata']['iterations']
    if iterations not in iterations_count:
        iterations_count[iterations] = 0
    iterations_count[iterations] += 1
print("Iterations count:")
for iterations, count in iterations_count.items():
    print(f"{iterations}: {count}")

Iterations count:
10: 40
